In [9]:
# Step 1: Load and Preprocess Product Data 
#We prepare the product list and assign weights based on prices (assuming cheaper items are more common).

import pandas as pd
import numpy as np

# Load and clean column headers
df = pd.read_excel("Market_Price_Comparison_2025.xlsx")
df.columns = df.columns.str.strip()  # Remove leading/trailing whitespace

# Now extract the product and price columns
products = df[['Supermarket A Product', 'Supermarket A Price']].copy()
products.columns = ['Product', 'Price']

# Drop duplicates and NaNs
products.dropna(subset=['Product', 'Price'], inplace=True)
products = products.drop_duplicates().reset_index(drop=True)

# Show cleaned data
products.head()


,Product,Price
0,Reis Yeşil Mercimek 1 Kg,129.95
1,Hasata Yozgat Sultani Yeşil Mercimek 1 Kg,135.95
2,Reis Kırmızı Mercimek 1 Kg,140.95
3,Reis Sarı Mercimek 1 Kg,222.95
4,Reis Royal Beluga Mercimeği 500 G,182.95


In [12]:
# Step 2: Calculate Item Popularity Based on Price (cheaper = more frequent)
# We invert prices to approximate item popularity (cheaper items = higher probability).

# Avoid division by zero and normalize
products['Inverse_Price'] = 1 / (products['Price'] + 1e-6)

# Normalize to get probabilities that sum to 1
products['Probability'] = products['Inverse_Price'] / products['Inverse_Price'].sum()

# Preview the resulting probabilities
products[['Product', 'Price', 'Probability']].head(10)

,Product,Price,Probability
0,Reis Yeşil Mercimek 1 Kg,129.95,0.000336
1,Hasata Yozgat Sultani Yeşil Mercimek 1 Kg,135.95,0.000321
2,Reis Kırmızı Mercimek 1 Kg,140.95,0.000310
3,Reis Sarı Mercimek 1 Kg,222.95,0.000196
4,Reis Royal Beluga Mercimeği 500 G,182.95,0.000238
5,Hasata Mardin Kırmızı Mercimek 1 Kg,69.95,0.000624
6,Superfresh Taze Fasulye 450 G,99.95,0.000437
7,Superfresh Bezelye 450 G,79.95,0.000546
8,Superfresh Ispanak 450 G,76.95,0.000567
9,Superfresh Elma Dilim Patates 1 Kg,199.95,0.000218


In [14]:
# Step 3: Simulate 10,000 Customers
#Each customer gets a random basket size (1 to 25), with items sampled using weighted probabilities.

import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# Number of customers
num_customers = 10000

# Create list of products and their associated probabilities
product_list = products['Product'].tolist()
probabilities = products['Probability'].tolist()

# Simulate transactions
transactions = []

for _ in range(num_customers):
    basket_size = np.random.randint(1, 26)  # Basket size between 1 and 25
    basket = np.random.choice(product_list, size=basket_size, replace=False, p=probabilities)
    transactions.append(basket.tolist())

# Preview first 5 transactions
transactions[:5]


[['Exotic Portakal Havuç Suyu 750 Cc',
  'Ot Dergisi',
  'Tamek Karışık Meyve Nektarı 1 Lt',
  'Ülker Laviva Selection Antep Fıstıklı Gofret 45 G',
  'Bağdat İsot Biber 80 G',
  'Mr.Muscle Süper Güç Çok Amaçlı Temizleyici 750 Ml',
  'Knorr Fırında Tavuk Çeşnisi Kekikli Fesleğenli Susamlı 29 G'],
 ['Yupo Jelly Portakal Halkası 70 G',
  'Eti Gold Çikolatalı Gofret 29 g',
  'Superfresh Milföy Hamuru 1 Kg',
  'Namet 7/24 Dana Macar Salam 60 G',
  'Tat Bezelye 830 G',
  'Dreamies Somonlu Kedi Ödül Maması 60 G',
  'Cumhuriyet Gazetesi',
  'Ot Dergisi',
  'Pastavilla Tel Şehriye 500 G',
  'Eti Burçak Kakao Kremalı Bisküvi 100 g',
  'Bağdat Kimyon Tuzluklu Cam 48 G',
  'Filiz Tel Şehriye 500 G',
  'Ülker Çikolatalı Gofret Maxi 47 G',
  'Garnier Çarpıcı Renkler 1.0 Ekstra Yoğun Siyah',
  'Pakmaya Mayalı Poğaça Harcı 35 G',
  'Fersan Üzüm Sirke 2 L',
  'Tamek Kayısı Nektarı 200 Ml',
  'Birgün Gazetesi',
  'Ülker Susamlı Çubuk Kraker 70 G',
  'Nestle Damak Ala Beyaz Gofret 30 G',
  'Hacı Şakir Ba

In [18]:
# Step 4: Store Transactions
#Format each customer’s basket into a dataset where each row is a transaction.

# Convert transactions to DataFrame
transaction_df = pd.DataFrame({
    'TransactionID': range(1, len(transactions) + 1),
    'Basket': [', '.join(basket) for basket in transactions]
})

# Preview the transaction dataset
transaction_df.head()


,TransactionID,Basket
0,1,"Exotic Portakal Havuç Suyu 750 Cc, Ot Dergisi,..."
1,2,"Yupo Jelly Portakal Halkası 70 G, Eti Gold Çik..."
2,3,First Sensations Tropikal Meyve Aromalı Sakız ...
3,4,"Lipton Ice Tea Şeftali Pet 1 L, Algida Maraş U..."
4,5,"Eti Popkek Mini Hindistan Cevizli Kek 180 G, A..."


In [20]:
#Step 5: Reuse or Load Synthetic Transactions
# Use the previously created transaction_df or load the saved transaction dataset.

#Step 6: Convert Data to Transaction Format
# We’ll convert each basket into a one-hot encoded format suitable for mlxtend's Apriori algorithm.


import pandas as pd
import numpy as np
import random

df = pd.read_excel("Market_Price_Comparison_2025.xlsx")
df.columns = df.columns.str.strip()

products = df[['Supermarket A Product', 'Supermarket A Price']].copy()
products.columns = ['Product', 'Price']
products.dropna(subset=['Product', 'Price'], inplace=True)
products = products.drop_duplicates().reset_index(drop=True)

# Calculate probabilities
products['Inverse_Price'] = 1 / (products['Price'] + 1e-6)
products['Probability'] = products['Inverse_Price'] / products['Inverse_Price'].sum()

# Simulate transactions
random.seed(42)
np.random.seed(42)
num_customers = 10000
product_list = products['Product'].tolist()
probabilities = products['Probability'].tolist()

transactions = []
for _ in range(num_customers):
    basket_size = np.random.randint(1, 26)
    basket = np.random.choice(product_list, size=basket_size, replace=False, p=probabilities)
    transactions.append(basket.tolist())


In [22]:
#Step 7: Run Apriori Algorithm Extract frequent itemsets using a support threshold

# Step 3: Run Apriori Algorithm
from mlxtend.frequent_patterns import apriori

# Convert to one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_array = te.fit_transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

# Apply Apriori with min support of 1%
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

# Sort by support descending
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# Show top frequent itemsets
frequent_itemsets.head(10)


,support,itemsets
359,0.0766,(Ülker Badem Kraker Sade 38 G)
369,0.0722,(Ülker Dankek Pöti Havuçlu Tarçınlı 35 G)
158,0.0703,(Indomie Tavuk Noodle 70 G)
157,0.0683,(Indomie Sebzeli Noodle 75 G)
380,0.0658,(Ülker Krispi Peynirli Çubuk Kraker 43 G)
10,0.0648,(Alpella Kakaolu Kaplamalı Kremalı Mini Rulo P...
99,0.0647,(Eti Gold Çikolatalı Gofret 29 g)
379,0.0642,(Ülker Krispi Baharatlı Çubuk Kraker 43 G)
365,0.0622,(Ülker Dankek 8 Kek Çilekli 55 G)
354,0.0621,(Ülker 8 Kek Orman Meyveli 55 G)


In [36]:
# Step 8: Generate Association Rules + IF Statements

# Filter to Top 50 Frequent Products
top_items = products.nlargest(50, 'Probability')['Product'].tolist()

# Create filtered transactions only containing those top items
filtered_transactions = [
    [item for item in basket if item in top_items] for basket in transactions
]

# One-Hot Encode the filtered transactions
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_array = te.fit_transform(filtered_transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

# Run Apriori with lower support and max length to save memory
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(df_encoded, min_support=0.001, use_colnames=True, max_len=3)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

# Generate association rules with lowered confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)
rules = rules.sort_values(by=['confidence', 'support'], ascending=[False, False])

# Display rules in IF-THEN format
if not rules.empty:
    for idx, row in rules.iterrows():
        antecedent = ', '.join(list(row['antecedents']))
        consequent = ', '.join(list(row['consequents']))
        print(f"IF a customer buys [{antecedent}] THEN they are likely to also buy [{consequent}]")
        print(f"   → Confidence: {row['confidence']:.2f}, Support: {row['support']:.3f}, Lift: {row['lift']:.2f}\n")
else:
    print("No association rules found with current thresholds.")

# Display top few rules as a DataFrame
rules.head()




IF a customer buys [Eti Gold Çikolatalı Gofret 29 g, İçim İçimino Laktozsuz Kakaolu Süt 180 Ml] THEN they are likely to also buy [Ülker Badem Kraker Sade 38 G]
   → Confidence: 0.34, Support: 0.001, Lift: 4.50

IF a customer buys [İçim İçimino Laktozsuz Kakaolu Süt 180 Ml, Ülker Badem Kraker Sade 38 G] THEN they are likely to also buy [Eti Gold Çikolatalı Gofret 29 g]
   → Confidence: 0.29, Support: 0.001, Lift: 4.55

IF a customer buys [Eti Gold Çikolatalı Gofret 29 g, Ülker Kremalı Sandviç Bisküvi 61 G] THEN they are likely to also buy [Ülker Badem Kraker Sade 38 G]
   → Confidence: 0.29, Support: 0.001, Lift: 3.77

IF a customer buys [Ülker Kremalı Sandviç Bisküvi 61 G, Ülker Badem Kraker Sade 38 G] THEN they are likely to also buy [Eti Gold Çikolatalı Gofret 29 g]
   → Confidence: 0.27, Support: 0.001, Lift: 4.10

IF a customer buys [Ülker Çikolatalı Gofret 36 G, Ülker Dankek 8 Kek Çilekli 55 G] THEN they are likely to also buy [Ülker Badem Kraker Sade 38 G]
   → Confidence: 0.24, 

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
74,"(Eti Gold Çikolatalı Gofret 29 g, İçim İçimino...",(Ülker Badem Kraker Sade 38 G),0.0029,0.0766,0.0010,0.344828,4.501666,1.0,0.000778,1.409400,0.780122,0.012739,0.290478,0.178941
75,"(İçim İçimino Laktozsuz Kakaolu Süt 180 Ml, Ül...",(Eti Gold Çikolatalı Gofret 29 g),0.0034,0.0647,0.0010,0.294118,4.545868,1.0,0.000780,1.325008,0.782681,0.014903,0.245288,0.154787
58,"(Eti Gold Çikolatalı Gofret 29 g, Ülker Kremal...",(Ülker Badem Kraker Sade 38 G),0.0045,0.0766,0.0013,0.288889,3.771395,1.0,0.000955,1.298531,0.738168,0.016291,0.229899,0.152930
60,"(Ülker Kremalı Sandviç Bisküvi 61 G, Ülker Bad...",(Eti Gold Çikolatalı Gofret 29 g),0.0049,0.0647,0.0013,0.265306,4.100558,1.0,0.000983,1.273047,0.759854,0.019034,0.214483,0.142699
110,"(Ülker Çikolatalı Gofret 36 G, Ülker Dankek 8 ...",(Ülker Badem Kraker Sade 38 G),0.0041,0.0766,0.0010,0.243902,3.184105,1.0,0.000686,1.221271,0.688764,0.012547,0.181181,0.128479


In [ ]:
#📘 Top 10 Association Rules with Business Insights
#1.
#Rule:
#If a customer buys Eti Gold Çikolatalı Gofret 29 g and İçim İçimino Laktozsuz Kakaolu Süt 180 Ml,
#Then they are likely to also buy Ülker Badem Kraker Sade 38 G
#Confidence: 34% — Lift: 4.50

#📈 Insight: These three snack products are often bought together, suggesting they're popular for on-the-go snacking or school/work lunches.

#💡 Action: Place them in a combo pack or create a "Snack Trio" shelf in convenience sections.

#2.
#Rule:
#If a customer buys İçim İçimino Kakaolu Süt and Ülker Badem Kraker,
#Then they are likely to also buy Eti Gold Gofret
#Confidence: 29% — Lift: 4.55

#📈 Insight: Customers grabbing a drink and a salty snack are also inclined toward a sweet bar.

#💡 Action: Offer a buy-2-get-1-free or bundled discount on these 3.

#3.
#Rule:
#If a customer buys Ülker Çokomel Marshmallow and Indomie Noodle,
#Then they are likely to also buy Ülker Badem Kraker
#Confidence: 24% — Lift: 3.11

#📈 Insight: Suggests evening snack + quick dinner pattern.

#💡 Action: Shelf together in late-evening aisles or near microwaves. Offer a "Netflix Night" snack combo.

#4.
#Rule:
#If a customer buys Halley Bisküvi and Kekstra Muffin,
#Then they are likely to also buy Indomie Chicken Noodle
#Confidence: 22% — Lift: 3.16

#📈 Insight: Sweet snacks are often paired with quick meals.

#💡 Action: Create "meal & dessert" kits targeting students or night-shift workers.

#5.
#ule:
#If a customer buys Ülker Dankek 8 Kek and Yupo Çokojelo,
#Then they are likely to also buy Dankek Havuçlu Tarçınlı
#Confidence: 22% — Lift: 3.05

#📈 Insight: Customers buying sweets often pick up more than one variety.

#💡 Action: Use this for flavor-variant cross-promotion or “Mix 3 for a Deal” campaigns.

#6.
#Rule:
#If a customer buys Eti Gold Gofret and Indomie Gurme Noodle,
#Then they are likely to also buy Ülker Badem Kraker
#Confidence: 22% — Lift: 2.85

#📈 Insight: Suggests cross-category purchases from different sections (sweet bar, noodle, savory snack).

#💡 Action: Bundle across categories — cross-department promotions.

#7.
#Rule:
#If a customer buys Ülker Dankek Duble Kek and Dankek Havuçlu Tarçınlı,
#Then they are likely to also buy Yupo Çokojelo
#Confidence: 20% — Lift: 3.37

#📈 Insight: Again reinforces the multi-dessert or variety-seeking behavior.

#💡 Action: Optimize assortment layout in bakery or snack aisle for easier flavor comparison.

#8.
#Rule:
#If a customer buys Ülker Susamlı Kraker and Indomie Noodle,
#Then they are likely to also buy Halley Bisküvi
#Confidence: 19% — Lift: 3.13

#📈 Insight: A clear meal + treat buying behavior.

#💡 Action: Position sweet biscuits near noodles — cross-sell in dinner sections.

#9.
#Rule:
#If a customer buys Kekstra Muffin and Yupo Çokojelo,
#Then they are likely to also buy Badem Kraker
#Confidence: 20% — Lift: 2.56

#📈 Insight: Sweet items are linked to salty cravings.

#💡 Action: Apply dynamic pricing when sweet and salty items are bought together — to increase average basket value.

#10.
#Rule:
#If a customer buys Eti Gold Gofret and Ülker Badem Kraker,
#Then they are likely to also buy İçim Kakaolu Süt
#Confidence: 13% — Lift: 3.68

#📈 Insight: A popular on-the-go snack + drink combo.

#💡 Action: Add these to a meal-deal shelf at checkout or near refrigerated drinks.

#📌 Conclusion
#These patterns reveal real opportunities in:

#   Product placement

#   Combo offers

#   Cross-category bundles

#   Personalized promotions (via loyalty apps)

In [42]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

# One-hot encode the filtered transactions
te = TransactionEncoder()
te_array = te.fit_transform(filtered_transactions)

# Create DataFrame: columns = products, rows = customers
df_wide_format = pd.DataFrame(te_array, columns=te.columns_)

# Add a TransactionID column (row = customer)
df_wide_format.insert(0, 'TransactionID', range(1, len(df_wide_format) + 1))

# Export to CSV (rows = customers, columns = products)
df_wide_format.to_csv("synthetic_transactions_wide_format.csv", index=False)
